In [ ]:
!pip uninstall -y transformers
!pip install -U transformers==4.40.2

Found existing installation: transformers 4.40.2
Uninstalling transformers-4.40.2:
  Successfully uninstalled transformers-4.40.2
  Using cached transformers-4.40.2-py3-none-any.whl.metadata (137 kB)
Using cached transformers-4.40.2-py3-none-any.whl (9.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.0 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.


In [ ]:
!pip install -U \
  bitsandbytes \
  transformers \
  peft \
  accelerate \
  datasets

  Using cached transformers-4.57.5-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.3 kB)
Using cached transformers-4.57.5-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-1.5B"

DATA_DIR = "/content"
TRAIN_FILE = os.path.join(DATA_DIR, "train.jsonl")
VAL_FILE = os.path.join(DATA_DIR, "val.jsonl")

ADAPTER_DIR = os.path.join(DATA_DIR, "adapters")
os.makedirs(ADAPTER_DIR, exist_ok=True)

MAX_SEQ_LEN = 1200

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
dataset = load_dataset(
    "json",
    data_files={
        "train": TRAIN_FILE,
        "validation": VAL_FILE,
    }
)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 900
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 100
    })
})


In [ ]:
def format_prompt(example):
    return (
        "### Instruction:\n"
        f"{example['instruction']}\n\n"
        "### Input:\n"
        f"{example['input']}\n\n"
        "### Response:\n"
        f"{example['output']}"
    )

In [ ]:
def tokenize_fn(example):
    text = format_prompt(example)
    tokens = tokenizer(
        text,
        truncation=True,
        max_length=MAX_SEQ_LEN,
        padding=False,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_ds = dataset.map(
    tokenize_fn,
    remove_columns=dataset["train"].column_names,
    batched=False,
)

print(tokenized_ds)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 900
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "v_proj"
    ],
)

model = get_peft_model(model, lora_config)

In [ ]:
def print_trainable_params(model):
    trainable = 0
    total = 0
    for _, param in model.named_parameters():
        total += param.numel()
        if param.requires_grad:
            trainable += param.numel()
    print(f"Trainable params: {trainable}")
    print(f"Total params: {total}")
    print(f"Trainable %: {100 * trainable / total:.2f}%")

print_trainable_params(model)

Trainable params: 2179072
Total params: 890795520
Trainable %: 0.24%


In [ ]:
!pip install -U transformers accelerate peft

In [ ]:
training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    fp16=False,
    optim="paged_adamw_8bit",
    report_to="none",
    run_name="qwen-qlora-go",
)

In [ ]:
from transformers import DataCollatorWithPadding
import torch

class MinimalCausalLMCollator(DataCollatorWithPadding):
    def __call__(self, features):
        labels = [f["labels"] for f in features]
        for f in features:
            f.pop("labels")

        batch = super().__call__(features)

        max_len = batch["input_ids"].shape[1]
        batch["labels"] = torch.tensor(
            [l + [-100] * (max_len - len(l)) for l in labels]
        )
        return batch

In [ ]:
data_collator = MinimalCausalLMCollator(tokenizer=tokenizer)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.175700
20,1.975900
30,2.132100
40,2.148500
50,2.057800
60,1.884100
70,1.761400
80,2.001300
90,1.923900
100,1.846900


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=675, training_loss=1.8912658366450557, metrics={'train_runtime': 8071.4405, 'train_samples_per_second': 0.335, 'train_steps_per_second': 0.084, 'total_flos': 1.385299421263872e+16, 'train_loss': 1.8912658366450557, 'epoch': 3.0})

In [ ]:
model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)

print(f"Adapter saved to {ADAPTER_DIR}")

Adapter saved to /content/adapters


In [48]:
!zip -r adapter.zip ./adapters/

  adding: adapters/ (stored 0%)
  adding: adapters/README.md (deflated 65%)
  adding: adapters/chat_template.jinja (deflated 71%)
  adding: adapters/tokenizer_config.json (deflated 89%)
  adding: adapters/special_tokens_map.json (deflated 69%)
  adding: adapters/adapter_model.safetensors (deflated 8%)
  adding: adapters/added_tokens.json (deflated 67%)
  adding: adapters/tokenizer.json (deflated 81%)
  adding: adapters/adapter_config.json (deflated 57%)
  adding: adapters/vocab.json (deflated 61%)
  adding: adapters/merges.txt (deflated 57%)
